<a href="https://colab.research.google.com/github/Shankar-Pankhania/Twitter_Sentiment_Analysis_on_Bitcoin/blob/main/count_sentiments_per_hour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect to Google Drive

In [1]:
from google.colab import drive 
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Dissertation/

Mounted at /content/drive/
/content/drive/My Drive/Dissertation


Import Package

In [2]:
import numpy as np
import pandas as pd
from google.colab import files

Read Dataset

In [3]:
df = pd.read_csv('bitcoin_tweets.csv')

Download Botometer

In [ ]:
pip install botometer

Detect bots

In [5]:
from botometer import Botometer

#samples 1000 tweets from the dataset
sample_entire_dataset = df.sample(n=1000)

rapid_api_key = '0dd6bfa718mshf0f360df07771b5p13cce4jsnae7a91cae654'

#Get your Twitter API credentials and enter them here
twitter_tokens = {
  'consumer_key' : 'ngUiFH7BGoDjoeRYFp6J4CISz',
  'consumer_secret' : 'sLGTulZAxLThdluHb7RyGTF08jpc75NowJkwj8DaNNG8P7zbeA',
  'access_key' : '2387367300-CsyGg6vQeV7reiGsOzFoRToHi7Jdskyie4Rf3GO',
  'access_secret' : 'J6ivsjOUFilDm7llUgvXe87Cwk8NEGZQyCS9wmgOanINU'
}

botometer = Botometer(mashape_key=rapid_api_key, **twitter_tokens, wait_on_ratelimit=True)

total_bots = []
updateUsernames = sample_entire_dataset['Username']

for i in updateUsernames:
  #validation for the users account
  try:
    #gets the botometer scores of the username
     usernames = botometer.check_account('@'+i)

  except:
    pass
 
  #checks to see if the username is a bot by comparing the overall score against 0.8
  if (usernames['raw_scores']['english']['overall'] >= 0.8):
    total_bots.append(i)

#set sentiment of bot tweet to NULL
for user in total_bots:
  df.loc[df['Username'] == user, ['sentiment']] = np.nan

Display list of all bots detected

In [10]:
total_bots

['binance_buddy',
 'TireHours',
 'Ronald_rol0',
 'BULLMAR21706445',
 'AcerboLivio',
 'bitcoinagile',
 'jenny_jen27',
 'undrfyr',
 'candidate7153',
 'RoyotPi',
 'BitcoinLifeClub',
 'Sneaksite',
 'HodlingBitcoin',
 'media_diamante',
 'CorieBouldin',
 'cryptow1re',
 'aqxvwy',
 'BittrexPrices',
 'coinpricenow',
 'shani_gujjr',
 'help_me_plzplz_',
 'crypto_AI_',
 'via_klare_123',
 'Serhot14',
 'BitcoinBolt',
 'bitoco_in',
 'Dl23Icy',
 'Pleznaofficial',
 'Raymond12590224',
 'LarryG73372822',
 'yilmazonur23',
 'NewsAlltime1',
 'chaintip',
 'thembi_mbatha1',
 'EtherFlair',
 'kellysiqili',
 'MotoringChat',
 'zeeshan17793755',
 'BTCstJournal',
 'JnHossein',
 'BTCstJournal',
 'CryptoNewsKing',
 'TomasEnrique92',
 'James45771815',
 'bitoco_in',
 'MichaelCMcKee',
 'Letho37',
 'Forexfr',
 'ladycrypt4',
 'bitcoinagile',
 'HackPosts',
 'CryptoCurrNews',
 'btc_blockbot',
 'is_acdc',
 'BURNERA56462776',
 'LatestcryptoN',
 'deesbarbara',
 'falxdsts',
 'Veronicaarthur0',
 'webnowcompany',
 'Chaudhrybobby1

Count the sentiments for each hour

In [11]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

count_sentiments_per_hour = df.groupby([pd.Grouper(key='Timestamp',freq='H'), df['sentiment']]).size().reset_index(name='count')

#this will convert rows as column headers and will be the new dataframe
df2 = count_sentiments_per_hour.pivot_table('count', ['Timestamp'], 'sentiment')
df2.rename(columns={-1 : 'count_negative', 0 : 'count_neutral', 1 : 'count_positive'}, inplace=True)

Count average compound each hour

In [12]:
count_compound_per_hour = df.groupby(pd.Grouper(key='Timestamp',freq='H'))['compound'].mean()
df2['compound_total'] = count_compound_per_hour.apply(lambda x: x)

Count average negative sentiment

In [13]:
filter_negative = df[df['sentiment'] == -1]
calculate_mean_negative_hourly = filter_negative.groupby(pd.Grouper(key='Timestamp',freq='H'))['compound'].mean()
df2['sent_negative'] = calculate_mean_negative_hourly.apply(lambda x: x)


Count average positive sentiment

In [14]:
filter_positive = df[df['sentiment'] == 1]
calculate_mean_positive_hourly = filter_positive.groupby(pd.Grouper(key='Timestamp',freq='H'))['compound'].mean()
df2['sent_positive'] = calculate_mean_positive_hourly.apply(lambda x: x)

Total volume of tweets for each hour

In [15]:
#total volume of tweets include bot tweets
total_volume_of_tweets = df.groupby(pd.Grouper(key='Timestamp',freq='H'))['Text'].count()
df2['total_volume_of_tweets'] = total_volume_of_tweets.apply(lambda x: x)
df2

sentiment,count_negative,count_neutral,count_positive,compound_total,sent_negative,sent_positive,total_volume_of_tweets
Timestamp,,,,,,,
2021-03-20 23:00:00,154,322,441,0.183093,-0.408458,0.484968,997
2021-03-21 00:00:00,133,342,483,0.196218,-0.392044,0.508789,1001
2021-03-21 01:00:00,162,377,400,0.130551,-0.442727,0.484292,997
2021-03-21 02:00:00,167,346,461,0.152785,-0.393613,0.473624,1003
2021-03-21 03:00:00,157,290,526,0.207234,-0.418376,0.521672,1001
...,...,...,...,...,...,...,...
2021-03-25 20:00:00,147,394,427,0.163823,-0.397025,0.516566,1003
2021-03-25 21:00:00,174,344,445,0.147562,-0.454155,0.507340,994
2021-03-25 22:00:00,136,409,437,0.187263,-0.403721,0.545909,1000


Download the dataframe

In [16]:
df2.to_csv('Data_to_hourly_intervals.csv')
files.download('Data_to_hourly_intervals.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>